In [5]:

from sqlalchemy import create_engine, text
import oracledb

import os 
from dotenv import load_dotenv
load_dotenv()

db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_sid = os.getenv('DB_SID') 

dsn = oracledb.makedsn(db_host, db_port, sid=db_sid)
connection_string = f"oracle+oracledb://{db_user}:{db_password}@{dsn}"

def get_db_connection():
    try:
        engine = create_engine(connection_string)
        return engine.connect()
    except Exception as e:
        print(f"Database connection error: {str(e)}")
        raise

In [6]:
def get_job_list(conn):
    try:
        query = text("SELECT JOBID, MAPID, MAPREF, FRQCD, TRGSCHM, TRGTBTYP, TRGTBNM, SRCSYSTM, STFLG, RECCRDT, RECUPDT, CURFLG, BLKPRCROWS FROM DWJOB")
        details_result = conn.execute(query).mappings().fetchall()
        return details_result
    except Exception as e:
        print(f"Error fetching job list: {str(e)}")
        raise


In [4]:
get_job_list(get_db_connection())

[{'jobid': 1, 'mapid': 1, 'mapref': 'DIM_ACCOUNT', 'frqcd': 'DL', 'trgschm': 'TRG', 'trgtbtyp': 'DIM', 'trgtbnm': 'DIM_ACCOUNT', 'srcsystm': 'CBS', 'stflg': 'A', 'reccrdt': datetime.datetime(2025, 3, 11, 3, 59, 17), 'recupdt': datetime.datetime(2025, 3, 11, 3, 59, 17), 'curflg': 'Y', 'blkprcrows': 50000}]

In [5]:
def get_parameter_mapping(conn):
    try:
        query=text("""SELECT PRTYP, PRCD, PRDESC, PRVAL, PRRECCRDT, PRRECUPDT
        FROM DWPARAMS""")        
        details_result = conn.execute(query).mappings().fetchall()
        return details_result
    except Exception as e:
        print(f"Error fetching job list: {str(e)}")
        raise


In [15]:
def get_parameter_mapping_datatype(conn):
    try:
        query=text("""SELECT  PRCD, PRDESC FROM DWPARAMS WHERE PRTYP = 'Datatype'""")        
        details_result = conn.execute(query).mappings().fetchall()
        return details_result
    except Exception as e:
        print(f"Error fetching job list: {str(e)}")
        raise


In [16]:
get_parameter_mapping_datatype(get_db_connection())



[{'prcd': 'test1', 'prdesc': 'Identifies alpha-numeric value of 1 Character.'},
 {'prcd': 'String1', 'prdesc': 'Identifies alpha-numeric value of 1 Character.'},
 {'prcd': 'String3', 'prdesc': 'Identifies alpha-numeric value of 3 Characters.'},
 {'prcd': 'String5', 'prdesc': 'Identifies alpha-numeric value of 5 Characters.'},
 {'prcd': 'String10', 'prdesc': 'Identifies alpha-numeric value of 10 Characters.'},
 {'prcd': 'String20', 'prdesc': 'Identifies alpha-numeric value of 20 Characters.'},
 {'prcd': 'String30', 'prdesc': 'Identifies alpha-numeric value of 30 Characters.'},
 {'prcd': 'String50', 'prdesc': 'Identifies alpha-numeric value of 50 Characters.'},
 {'prcd': 'String100', 'prdesc': 'Identifies alpha-numeric value of 100 Characters.'},
 {'prcd': 'String250', 'prdesc': 'Identifies alpha-numeric value of 250 Characters.'},
 {'prcd': 'String4000', 'prdesc': 'Identifies alpha-numeric value of 4000 Characters.'},
 {'prcd': 'Numeric1', 'prdesc': 'Identifies numeric value of 1 digit.

In [3]:
def get_parameter_mapping(conn):
    try:
        cursor = conn.cursor()
        type_value="Datatype"
        query = "SELECT PRTYP, PRCD, PRDESC, PRVAL, PRRECCRDT, PRRECUPDT FROM DWPARAMS WHERE PRTYP = :type"
        cursor.execute(query, {"type": type_value})
        
        # Get column names from cursor description
        columns = [col[0] for col in cursor.description]
        
        # Fetch all rows and convert to dictionaries
        result = []
        for row in cursor.fetchall():
            result.append(dict(zip(columns, row)))
            
        cursor.close()
        return result
    except Exception as e:
        print(f"Error fetching parameter mapping: {str(e)}")
        raise

In [4]:
get_parameter_mapping(get_db_connection())


NameError: name 'get_db_connection' is not defined

In [3]:
# def add_parameter_mapping(conn, type, code, desc, value):
#     try:
#         query = text("""
#             INSERT INTO DWPARAMS (PRTYP, PRCD, PRDESC, PRVAL, PRRECCRDT, PRRECUPDT)
#             VALUES (:type, :code, :desc, :value, sysdate, sysdate)
#         """)
#         conn.execute(query, {"type": type, "code": code, "desc": desc, "value": value})
#         conn.commit()
#         return "Parameter mapping added successfully."
#     except Exception as e:
#         print(f"Error adding parameter mapping: {str(e)}")
#         raise

import oracledb
def create_oracle_connection():
    try:
        connection = oracledb.connect(
            user="map", 
            password="mapper", 
            dsn="172.16.20.69:1521/demodb"
        )
        print("Oracle connection established successfully")
        return connection
    except Exception as e:
        print(f"Error establishing Oracle connection: {str(e)}")
        raise



def add_parameter_mapping(conn, type, code, desc, value):
    try:
        cursor = conn.cursor()
        query = """
            INSERT INTO DWPARAMS (PRTYP, PRCD, PRDESC, PRVAL, PRRECCRDT, PRRECUPDT)
            VALUES (:1, :2, :3, :4, sysdate, sysdate)
        """
        cursor.execute(query, [type, code, desc, value])
        conn.commit()
        cursor.close()
        return "Parameter mapping added successfully."
    except Exception as e:
        print(f"Error adding parameter mapping: {str(e)}")
        raise

In [4]:
add_parameter_mapping(create_oracle_connection(),"test","varchar","tesing the funtion ","varchar2(20)")

Oracle connection established successfully


'Parameter mapping added successfully.'

In [9]:
add_parameter_mapping(get_db_connection(),"Datatype1","test11","Identifies alpha-numeric value of 1 Character.","Varchar2(129)")

'Parameter mapping added successfully.'

In [16]:
def get_error_massage(conn, ref):
    try:
        query = text("""
            SELECT errmsg FROM DWMAPERR WHERE DWMAPERR.MAPDTLID = :map_detail_id ORDER  BY DWMAPERR.MAPERRID DESC FETCH FIRST 1 ROWS ONLY 
        """)
        result=conn.execute(query, {"map_detail_id": ref}).mappings().fetchall()
        
        if not result :
            return "Logic is Verified"
        
        return result[0]['errmsg']
    except Exception as e:
        print(f"Error adding parameter mapping: {str(e)}")
        raise

In [17]:
get_error_massage(get_db_connection(),48)

'ORA-00907: missing right parenthesis'

In [21]:
def get_error_messages(conn, map_detail_ids):
    """
    map_detail_ids: list of references from detail mapping table
    Returns: dictionary mapping each id to its error message
    """
    try:
        result_dict = {}
        cursor = conn.cursor()
        query = """
            SELECT errmsg FROM DWMAPERR 
            WHERE DWMAPERR.MAPDTLID = :1 
            ORDER BY DWMAPERR.MAPERRID DESC 
            FETCH FIRST 1 ROWS ONLY
        """
        
        for map_detail_id in map_detail_ids:
            cursor.execute(query, [map_detail_id])
            
            # Fetch one row
            row = cursor.fetchone()
            
            if not row:
                result_dict[map_detail_id] = "Logic is Verified"
            else:
                result_dict[map_detail_id] = row[0]  # Return the first column value
        
        cursor.close()
        return result_dict
        
    except Exception as e:
        print(f"Error getting error messages: {str(e)}")
        raise

In [24]:
from dotenv import load_dotenv
load_dotenv()

db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_sid = os.getenv('DB_SID')
# print(db_host,db_user,db_password,db_sid,db_port)


dsn = oracledb.makedsn(db_host, db_port, sid=db_sid)
connection_string = f"oracle+oracledb://{db_user}:{db_password}@{dsn}"

# Establish connection outside the function (do this once)
def create_oracle_connection():
    try:
        connection = oracledb.connect(
            user="map", 
            password="mapper", 
            dsn="172.16.20.69:1521/demodb"
        )
        print("Oracle connection established successfully")
        return connection
    except Exception as e:
        print(f"Error establishing Oracle connection: {str(e)}")
        raise


In [25]:
get_error_messages(create_oracle_connection(),[48,106])

Oracle connection established successfully


{48: 'ORA-00907: missing right parenthesis',
 106: 'ORA-00904: "COD_ACCT_N1": invalid identifier'}